In [ ]:
# MedVQA directory path
MEDVQA_DIR_PATH = "/content/drive/MyDrive/MedVQA"

# Hierarchichal + CoAtt. main.py script path
HIE_MAIN_PY_PATH = "/content/drive/MyDrive/MedVQA/hierarchical/main.py"

In [ ]:
!python3 /content/drive/MyDrive/MedVQA/hierarchical/main.py

/content/drive/MyDrive/MedVQA/hierarchical/main.py
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 256MB/s]
200


In [ ]:
!pip install torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
checkpoint_path = "/content/drive/MyDrive/MedVQA/Med-VQA/checkpoint.pth"

In [ ]:
import torch

model = torch.load(checkpoint_path)
#print(model)

In [ ]:
import torch.nn as nn

class TempModel(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(2, 1000, (64, 64))
        self.conv1 = nn.Conv2d(3, 32, (448, 448))
    def forward(self, inp1, inp2):
       # self.conv1(inp1)
        #self.conv1(inp2)
        self.inp1=inp1
        self.inp2=inp2
        return conv1

modelMain = TempModel()
modelMain.load_state_dict(model, strict=False)
modelMain.eval()

TempModel(
  (conv1): Conv2d(3, 32, kernel_size=(448, 448), stride=(1, 1))
)

In [ ]:
!cp /content/drive/MyDrive/MedVQA/hierarchical/_config.py .

In [ ]:
!cp /content/drive/MyDrive/MedVQA/hierarchical/_utils.py .
!cp /content/drive/MyDrive/MedVQA/hierarchical/main.py .
!cp /content/drive/MyDrive/MedVQA/hierarchical/trainer.py .
!cp /content/drive/MyDrive/MedVQA/hierarchical/model.py .

In [ ]:
from _config import Config as config
import trainer
import _utils
import model

In [ ]:
answers_dict = _utils.Datamake.make_ans_dict(config.TRAIN_DATA_DICT_PATH)


In [ ]:
train_loader, pad_idx, vocab = _utils.Datamake.get_loader(image_path=config.TRAIN_IMG_PATH, vqa_path=config.TRAIN_DATA_DICT_PATH)


In [ ]:
k = trainer.Trainer.accuracy(modelMain, train_loader, answers_dict, config)
k

NameError: ignored

In [ ]:
# from future import print_function
# import torch 
# from _config import Config as config
# import model
# import trainer
# import _utils
from pathlib import Path
import sys
import time
import numpy as np
torch.backends.cudnn.benchmark = True

path = Path(config.CHECKPOINT_PATH)
# path.mkdir(parents=True, exist_ok=True)
# stats_file = open(path / 'stats.txt', 'a', buffering=1)
# print(' '.join(sys.argv))
# print(' '.join(sys.argv), file=stats_file)

# answers_dict = _utils.Datamake.make_ans_dict(config.TRAIN_DATA_DICT_PATH)
answers_dict = _utils.Datamake.make_ans_dict(config.TEST_DATA_DICT_PATH)

train_loader, pad_idx, vocab = _utils.Datamake.get_loader(image_path=config.TRAIN_IMG_PATH, vqa_path=config.TRAIN_DATA_DICT_PATH)
valid_loader, val_pad_idx, val_vocab = _utils.Datamake.get_loader_val(vocab=vocab, image_path=config.TEST_IMG_PATH, vqa_path=config.TEST_DATA_DICT_PATH)

num_classes = len(answers_dict)
vocab_len = len(vocab[0])
coattention = model.CoAttention(num_embeddings=vocab_len, num_classes=num_classes, embed_dim=512, k=64).to(config.DEVICE)    
visualfeatures = model.VisualFeatures(device=config.DEVICE).to(config.DEVICE)
stud = model.Identity(coattention, visualfeatures, vocab_len)
print(stud)
optimizer = torch.optim.Adam(stud.parameters(),lr=config.LR)

def accuracy(model, loader, answers_dict, optimizer):
  model.eval()
  ssum=0
  c=0
  for data in loader:
    image = data[0] #shape : [batch, H, W, C]
    question = data[1]
    answer = data[2]
    batch_size = image.shape[0]
    c+=batch_size

    image = image.to('cuda')
    question = question.long().to('cuda')

    image = image.permute(0,3,1,2)
    ans_list = []
    for i in answer:
      ans_list.append(answers_dict[i])
    
    target = torch.Tensor(ans_list).long().to('cuda')
    output = model(image, question)
    output = torch.softmax(output, dim=-1)
    output = torch.argmax(output, dim=-1)
    # output = output.view(batch_size,-1)
    target = target.view(-1)
    ssum += torch.sum(output==target)
  print((ssum/c).item())
  return ssum/c

# def accuracy(model, valid_loader, answers_dict, config):
#         iterator = 1
#         with torch.no_grad():
#             k=0
#             for n, i in enumerate(valid_loader):
#                 print("Iteration: ", iterator)
#                 print("valid_loader LEN = ", len(valid_loader))
#                 images = i[0].permute(0, 3, 1, 2).to(config.DEVICE)
#                 question = i[1].long().to(config.DEVICE)
#                 answers = i[2]

#                 target=[]
#                 tmp=[]

#                 iterator += 1

#                 for i in answers:
#                     tmp.append(answers_dict[i])
                
#                 target = torch.Tensor(target).long().to(config.DEVICE)
#                 out = model(images, question)
#                 out = torch.softmax(out, -1)
#                 out = torch.argmax(out, dim=1)
#                 tmp = np.array(tmp)
#                 print("tmp = ", tmp)
#                 out = np.array(out.tolist())
#                 k += np.sum(tmp==out)
#                 print("out = ", out)
#                 print("k = ", k)


#         return k

if (path / 'checkpoint.pth').is_file():
        ckpt = torch.load(config.CHECKPOINT_PATH +"/"+ 'checkpoint.pth',
                          map_location='cpu')
        start_epoch = ckpt['epoch']
        print("Epoch:",ckpt['epoch'])
        stud.load_state_dict(ckpt['model'])
        print("Model:",ckpt['model'])
        optimizer.load_state_dict(ckpt['optimizer'])
        print("Optimizer",ckpt['optimizer'])
else:
#         start_epoch = config.START_EPOCH 
        pass
    
stud = stud.to(config.DEVICE)
# config.START_EPOCH = start_epoch
# print("ok")
# criterion = torch.nn.CrossEntropyLoss()
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3,verbose=True)

# trainer.Trainer.train(stud, train_loader, valid_loader, answers_dict, optimizer, criterion, scheduler, config, stats_file)
print("Stud",stud)
print(accuracy(stud, valid_loader, answers_dict, config))

Streaming output truncated to the last 5000 lines.
         3.9732e-10,  4.5206e-03, -1.8991e-03,  1.8123e-03,  1.1488e-02,
        -5.1125e-03,  4.2108e-04,  1.2669e-02,  2.4040e-04, -7.0472e-04,
        -9.8992e-03,  6.9213e-03,  4.6383e-04, -1.9211e-05, -1.1271e-03,
         1.1631e-02, -3.5963e-04, -6.7111e-03, -1.0692e-02,  1.0197e-03,
        -4.3728e-04,  3.5631e-04,  7.2596e-04, -9.5394e-04,  4.8494e-03,
         3.4084e-03, -2.2478e-03,  3.7668e-06,  1.4107e-03,  1.7874e-03,
        -2.1300e-03,  4.9555e-03,  3.8234e-03, -6.4561e-05,  2.1142e-07,
         1.3440e-04, -3.1402e-02, -2.2729e-02, -1.9935e-03,  2.0786e-03,
        -2.9322e-02,  5.6052e-45,  1.1354e-04, -7.4477e-04,  4.6760e-02,
         1.1779e-05,  1.7819e-04,  4.4818e-04, -4.9886e-03, -5.6052e-45,
         7.4051e-05,  5.0481e-04, -6.9164e-03, -1.6908e-02, -2.3615e-04,
        -7.3602e-04, -4.9988e-04,  1.0696e-03,  1.2461e-03, -9.7701e-03,
         6.9652e-04,  1.4355e-03,  2.9439e-03,  1.1712e-02, -2.1789e-03,
